In [72]:
%matplotlib nbagg

import numpy as np
import matplotlib.pyplot as plt
import math

In [73]:
n = 6
num_samples = 600
MU_0 = 0
MU_1 = 1
SIGMA = 1

In [74]:
def generate_time_series(MU_0, MU_1, SIGMA, N, NUM_SAMPLES):
    NUM_SAMPLES = int(NUM_SAMPLES / 2)
    TS = np.concatenate(( np.random.normal(MU_0,SIGMA,(NUM_SAMPLES, N)),np.random.normal(MU_1,SIGMA,(NUM_SAMPLES, N))))
    return TS

In [75]:
TS = generate_time_series(0, 1, 1, n, num_samples)
TS.shape

(600, 6)

In [76]:
mu_bar = (1/n)*np.sum(TS)
sigma_cap = (1/n)*np.sum(np.einsum('ij,ij->i', (TS-MU_0), (TS-MU_0)))

In [77]:
mu_bar, sigma_cap

(308.41793518727326, 907.34694986388695)

In [78]:
mle_h0 = (2*math.pi)^

SyntaxError: invalid syntax (<ipython-input-78-f3ca4866c22c>, line 1)